# Generate composite galaxy

Example for generating a composite galaxy
photometry. This example will:
- build two parametric "galaxies" (see make_sfzh)
- calculate spectral luminosity density of each
TODO: add image creation

In [1]:
from synthesizer.grid import Grid
from synthesizer.parametric.morphology import Sersic2D
from synthesizer.parametric.sfzh import SFH, ZH, generate_sfzh, generate_instant_sfzh
from synthesizer.galaxy import ParametricGalaxy as Galaxy
from synthesizer.filters import UVJ

from unyt import yr, Myr, kpc, mas
import matplotlib.pyplot as plt
import numpy as np

In [2]:
resolution = 0.1 * kpc
npix = 25

grid_dir = '../../../tests/test_grid'
grid_name = 'test_grid'
grid = Grid(grid_name, grid_dir=grid_dir)

filter_collection = UVJ(new_lam=grid.lam)

## Disk

Define morphology

In [3]:
morphology_parameters = {'r_eff': 1. * kpc, 'n': 1., 'ellip': 0.5, 'theta': 35.}
morph = Sersic2D(morphology_parameters)

Define the parameters of the star formation and metal enrichment histories

In [4]:
sfh_p = {'duration': 10 * Myr}
Z_p = {'log10Z': -2.0}  # can also use linear metallicity e.g. {'Z': 0.01}
stellar_mass = 1E8

Define the functional form of the star formation and metal enrichment histories

In [5]:
sfh = SFH.Constant(sfh_p)  # constant star formation
Zh = ZH.deltaConstant(Z_p)  # constant metallicity

Get the 2D star formation and metal enrichment history for the given SPS grid. This is (age, Z).

In [6]:
sfzh = generate_sfzh(grid.log10ages, grid.metallicities, sfh, Zh, stellar_mass=stellar_mass)

Initialise Galaxy object

In [7]:
disk = Galaxy(morph=morph, sfzh=sfzh)

Generate stellar spectra

In [8]:
disk.get_stellar_spectra(grid)

Generate broadband luminosities

In [9]:
sed = disk.spectra['stellar'].get_broadband_luminosities(filter_collection)

Make images

In [12]:
images = disk.make_images('stellar', filtercollection=filter_collection, 
                          resolution=resolution, npix=npix)
# images.plot_rgb()

TypeError: Image.__init__() got an unexpected keyword argument 'resolution'

In [ ]:
print(disk)

## Bulge

In [ ]:
morphology_parameters = {'r_eff': 1. * kpc, 'n': 4.}
morph = Sersic2D(morphology_parameters)

Define the parameters of the star formation and metal enrichment histories

In [ ]:
stellar_mass = 1E9
sfzh = generate_instant_sfzh(
    grid.log10ages, grid.metallicities, 10., 0.01, stellar_mass=stellar_mass)

bulge = Galaxy(morph=morph, sfzh=sfzh)

bulge.get_stellar_spectra(grid)

sed = bulge.spectra['stellar'].get_broadband_luminosities(filter_collection)

Make images

In [ ]:
images = bulge.make_images('stellar', filtercollection=UVJ, resolution=resolution, npix=npix)

In [ ]:
print(bulge)

## Total

In [ ]:
total = disk + bulge

print(total)

In [ ]:
images = bulge.make_images('stellar', filter_collection, resolution, npix=npix)

images.plot_rgb(['J', 'V', 'U'])

total = disk.images['stellar'].rgb_img + bulge.images['stellar'].rgb_img

total /= np.max(total)

plt.figure()
plt.imshow(total, origin='lower', interpolation='nearest')
plt.show()
